In this notebook, we train a model using the orientation data of people during fitness exercises. The model tries to predict whether the person is in a vertical or a horizontal position.

In [1]:
import sys
import os
import subprocess
import zipfile
import numpy as np

from oodles import Framework
from oodles import Signal
from oodles import monitor
from oodles import ModelSignal, AnnotationMethod, Anomaly, DataDriftAlgo

from dataset import input_to_dataset_transformation, read_json, write_json, KpsDataset
from pushup_signal import pushup_signal
from contextlib import redirect_stdout

import tensorflow as tf
import joblib
import json

Download dataset from remote

In [2]:
data_dir = "data"
remote_url = "https://oodles-dev-training-data.s3.amazonaws.com/data.zip"
orig_training_file = 'data/training_data.json'
if not os.path.exists(data_dir):
    try:
        file_downloaded_ok = subprocess.check_output("wget " + remote_url, shell=True)
    except:
        print("Could not load training data")
    with zipfile.ZipFile("data.zip", 'r') as zip_ref:
        zip_ref.extractall("./")

    full_training_data = read_json(orig_training_file)
    np.random.seed(1)
    np.random.shuffle(full_training_data)
    reduced_training_data = full_training_data[0:1000]
    write_json(orig_training_file, reduced_training_data)

In [3]:
real_world_test_cases = 'data/real_world_testing_data.json'
golden_testing_file = 'data/golden_testing_data.json'
annotation_args = {'master_file': 'data/master_annotation_data.json'}

# Defining the egde-case signal
pushup_edge_case = Signal("Pushup", pushup_signal)
inference_batch_size = 1

### Training with Logistic Regression (LR)

In [4]:
from model_logistic_regression import get_accuracy_lr, train_model_lr
train_model_lr('data/training_data.json', 'version_0')

Training on:  data/training_data.json  which has  1000  data-points
Trained model exists. Skipping training again.


Next, we evaluate the model on our golden testing dataset.

In [5]:
get_accuracy_lr(golden_testing_file, 'version_0')

Evaluating on  15731  data-points


0.8586231008836056

We observe that the testing accuracy of the model is quite low. On investigating further, we realize that it is because all pushup signals are being classified as "vertical" orientation. Next, we will define the oodles config with edge-case check for Pushup signals and also pass our training and evaluation arguments

In [6]:
cfg = {
    # Define your signal to identify edge cases
    "checks": [{
        'type': Anomaly.EDGE_CASE, 
        "signal_formulae": pushup_edge_case
    },
    {
        'type': Anomaly.DATA_DRIFT,
        'algorithm': DataDriftAlgo.DDM  
    },
    {
        'type': Anomaly.CONCEPT_DRIFT,
        'algorithm': DataDriftAlgo.DDM  
    }],
    "data_identifier": "id",
    "batch_size": inference_batch_size,

    # Connect training pipeline to annotate data and retrain the model
    "training_args": {
        "data_transformation_func": input_to_dataset_transformation,  
        "annotation_method": {"method": AnnotationMethod.MASTER_FILE, "args": annotation_args}, 
        "training_func": train_model_lr, 
        "fold_name": 'oodles_smart_data',  
        "orig_training_file": orig_training_file,  
    },

    # Connect evaluation pipeline to test retrained model against original model
    "evaluation_args": {
        "inference_func": get_accuracy_lr,
        "golden_testing_dataset": golden_testing_file,
        "metrics_to_check": ['accuracy']
    }
}

In [7]:
framework_lr = Framework(cfg)

@monitor(framework_lr)
def model_predict(model, inputs):
    return model.predict(inputs['data'])

Deleting the folder:  oodles_smart_data


In [8]:
testing_dataset = KpsDataset(real_world_test_cases, normalization=True)
X_test, y_test, id = testing_dataset.load_x_y_from_data()
pred_classes = []
model = joblib.load("trained_models_lr/" + 'version_0')
for i,elem in enumerate(X_test):

    # Do model prediction
    inputs = {"data": elem.reshape(1, -1), "id": [id[i]]}
    preds, idens = model_predict(model, inputs)

    # Attach Ground Truth
    framework_lr.attach_ground_truth({'id': idens, 'gt': [y_test[i]]})

    # Retrain only once
    if framework_lr.version > 1:
        break

50  edge-cases collected out of  76  inferred samples
100  edge-cases collected out of  145  inferred samples
150  edge-cases collected out of  224  inferred samples
200  edge-cases collected out of  296  inferred samples
250  edge-cases collected out of  367  inferred samples
Kicking off re-training
251 data-points selected out of 371
Training on:  oodles_smart_data/1/training_dataset.json  which has  2255  data-points
Trained model exists. Skipping training again.
Model retraining done...
Generating comparison report...
Training on:  data/training_data.json  which has  1000  data-points
Trained model exists. Skipping training again.
Evaluating on  15731  data-points
Evaluating on  15731  data-points
---------------------------------------------
---------------------------------------------
Old model accuracy:  0.8586231008836056
Retrained model accuracy (ie 251 smartly collected data-points added):  0.94386879410082
---------------------------------------------
----------------------

In the comparison report above, we can see how oodles improved the model performance by detecting edge-cases and retraining the model under-the-hood. Further, Oodles is agnostic to the model type and training functions. To illustrate this, we again train our orientation classification model, but this time with Deep Neural Networks.

### Training using Deep Neural Network

In [9]:
from model_dnn import get_accuracy_dnn, train_model_dnn
train_model_dnn('data/training_data.json', 'version_0')

Training on:  data/training_data.json  which has  1000  data-points
Trained model exists. Skipping training again.


Next, we get the model accuracy on testing dataset, which is again low due to misclassification of Pushup signals.

In [10]:
get_accuracy_dnn(golden_testing_file, 'version_0')

2022-12-08 15:24:05.033405: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Evaluating on  15731  data-points
492/492 [==============================] - 1s 657us/step


0.23634861102282118

Update the Oodles config with new training workflows and checks. Let's also add a check for edge-cases when model confidence is low (because why not!).

In [11]:
# Whenever model confidence is <0.8, identify it as an edge-case 
low_conf_edge_case = Signal(ModelSignal.BINARY_ENTROPY_CONFIDENCE, 
                is_model_signal=True, extra_args={'conf_threshold': 0.8})

cfg['checks'][0].update({"signal_formulae": (pushup_edge_case | low_conf_edge_case)})
cfg['training_args'].update({'training_func': train_model_dnn})
cfg['evaluation_args'].update({'inference_func': get_accuracy_dnn})

In [12]:
framework_dnn = Framework(cfg)

@monitor(framework_dnn)
def model_predict(model, inputs):
    with open('evaluation_logs.txt', 'w') as f:
        with redirect_stdout(f):
            return model.predict(inputs['data'])

Deleting the folder:  oodles_smart_data


In [13]:
model_dir = 'trained_models_dnn/'
model_save_name = 'version_0'
real_world_dataset = KpsDataset(
    real_world_test_cases, batch_size=inference_batch_size, shuffle=False, augmentations=False, is_test=True
)
model = tf.keras.models.load_model(model_dir + model_save_name)
gt_data = read_json(annotation_args['master_file'])
all_gt_ids = [x['id'] for x in gt_data]

for i,elem in enumerate(real_world_dataset):

    # Do model prediction
    preds, idens = model_predict(model, {"data": elem[0]["data"], "id": elem[0]["id"]})

    # Attach ground truth
    this_elem_gt = [gt_data[all_gt_ids.index(x)]['gt'] for x in elem[0]['id']]
    framework_dnn.attach_ground_truth({'id': idens, 'gt': np.array(this_elem_gt)})

    # Retrain only once
    if framework_dnn.version > 1:
        break

50  edge-cases collected out of  196  inferred samples
100  edge-cases collected out of  409  inferred samples
150  edge-cases collected out of  599  inferred samples
200  edge-cases collected out of  805  inferred samples
250  edge-cases collected out of  972  inferred samples
Kicking off re-training
251 data-points selected out of 974
Training on:  oodles_smart_data/1/training_dataset.json  which has  2255  data-points
Trained model exists. Skipping training again.
Model retraining done...
Generating comparison report...
Training on:  data/training_data.json  which has  1000  data-points
Trained model exists. Skipping training again.
Evaluating on  15731  data-points
492/492 [==============================] - 0s 370us/step
Evaluating on  15731  data-points
492/492 [==============================] - 0s 371us/step
---------------------------------------------
---------------------------------------------
Old model accuracy:  0.23634861102282118
Retrained model accuracy (ie 251 smartly 